<a href="https://colab.research.google.com/github/SumeyyeSuslu/regression/blob/main/tecWorking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark
!pip install construct

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http

In [ ]:
      #ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZzZ1`1

In [ ]:
!7z x DummyGeometryforPHDwork.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 30742711 bytes (30 MiB)

Extracting archive: DummyGeometryforPHDwork.7z
--
Path = DummyGeometryforPHDwork.7z
Type = 7z
Physical Size = 30742711
Headers Size = 1602
Method = LZMA2:24
Solid = +
Blocks = 2

  0%      9% 26 - DummyGeometry/mach0.200_aoa10. . 00_aoa10.00_beta0.00-POINT.dat                                                                          20% 35 - DummyGeometry/mach0.200_aoa6.00 . _mach0.200_aoa6.00_beta0.00.plt                                                               

In [ ]:
import os
file_paths = [] 
for root, dirs, files in os.walk("./DummyGeometry"):
    for file in files:
        if file.endswith(".plt"):
             file_paths.append(os.path.join(root, file))
print(file_paths)

['./DummyGeometry/mach1.000_aoa0.00_beta0.00/Tecplot_mach1.000_aoa0.00_beta0.00.plt', './DummyGeometry/mach0.600_aoa4.00_beta0.00/Tecplot_mach0.600_aoa4.00_beta0.00.plt', './DummyGeometry/mach0.600_aoa6.00_beta0.00/Tecplot_mach0.600_aoa6.00_beta0.00.plt', './DummyGeometry/mach1.000_aoa4.00_beta0.00/Tecplot_mach1.000_aoa4.00_beta0.00.plt', './DummyGeometry/mach0.800_aoa8.00_beta0.00/Tecplot_mach0.800_aoa8.00_beta0.00.plt', './DummyGeometry/mach0.200_aoa6.00_beta0.00/Tecplot_mach0.200_aoa6.00_beta0.00.plt', './DummyGeometry/mach0.800_aoa6.00_beta0.00/Tecplot_mach0.800_aoa6.00_beta0.00.plt', './DummyGeometry/mach1.000_aoa6.00_beta0.00/Tecplot_mach1.000_aoa6.00_beta0.00.plt', './DummyGeometry/mach1.000_aoa8.00_beta0.00/Tecplot_mach1.000_aoa8.00_beta0.00.plt', './DummyGeometry/mach0.800_aoa10.00_beta0.00/Tecplot_mach0.800_aoa10.00_beta0.00.plt', './DummyGeometry/mach0.200_aoa4.00_beta0.00/Tecplot_mach0.200_aoa4.00_beta0.00.plt', './DummyGeometry/mach0.600_aoa8.00_beta0.00/Tecplot_mach0.600_

In [ ]:
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import rand
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vector
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [ ]:

import re
import numpy as np
import pltreader as plt

dataset = []
for item in file_paths:
  file_content={}
  pat = re.compile("mach(.*)_aoa(.*)_beta(.*)/")
  obj = pat.search(item)
  mach = float(obj.group(1))
  aoa = float(obj.group(2))
  beta = float(obj.group(3))
  with open( item, 'rb') as f:
    bytelist = f.read()
  header = plt.read_header(bytelist)
  data = plt.read_data(bytelist,header)
  for zone_id in range (len(data['Zones'])):
    zone_shape = data['Zones'][zone_id]['Pressure'].shape
    data['Zones'][zone_id]['mach']=mach*np.ones(zone_shape, dtype = 'float32')
    data['Zones'][zone_id]['aoa']=aoa*np.ones(zone_shape, dtype = 'float32')
    data['Zones'][zone_id]['beta']=beta*np.ones(zone_shape, dtype = 'float32')


  file_content['data'] = data
  dataset.append(file_content)

In [ ]:
cols=['mach','aoa','beta']+header['VarNames']
zid=0
m=[]
for fid in range(len(file_paths)):
  t=[]
  for col in cols:
    t.append(dataset[fid]['data']['Zones'][zid][col])
  m.append(np.vstack(t))
s=np.hstack(m).transpose()

In [ ]:
# dataset[file_id]['data']['Zones'][zone_id][var_name])

In [ ]:
df=spark.createDataFrame(s.tolist(),cols)

In [ ]:
df.limit(15).toPandas()

,mach,aoa,beta,CoordinateX,CoordinateY,CoordinateZ,X Component Wall Shear,Pressure,Pressure Coefficient,Y Component Wall Shear,Z Component Wall Shear
0,1.0,0.0,0.0,11.741610,-1.658558,0.277781,213.831421,12711.195312,0.179342,-13.474180,36.881374
1,1.0,0.0,0.0,11.738341,-1.697367,0.270796,238.749771,-1151.620972,-0.016248,-25.190208,3.711837
2,1.0,0.0,0.0,11.705114,-1.663811,0.269339,227.467361,1780.602661,0.025122,-29.553242,4.312670
3,1.0,0.0,0.0,11.774469,-1.584260,0.253532,160.542801,-32521.728516,-0.458848,2.522774,-3.912574
4,1.0,0.0,0.0,11.839085,-1.553269,0.251952,165.042542,-34348.320312,-0.484620,3.872399,2.740007
5,1.0,0.0,0.0,11.764987,-1.520234,0.249566,168.176559,-33003.746094,-0.465649,3.399415,-0.360350
6,1.0,0.0,0.0,11.745173,-1.656447,0.262674,184.745331,-29742.916016,-0.419642,-2.657237,-19.765045
7,1.0,0.0,0.0,11.768639,-1.693918,0.265840,192.975708,-30481.328125,-0.430060,-2.042527,-23.169786
8,1.0,0.0,0.0,11.790966,-1.668062,0.260443,160.722336,-32474.470703,-0.458182,-1.928058,-10.442727
9,1.0,0.0,0.0,11.765932,-1.685940,0.278404,220.446304,11685.561523,0.164871,-22.545374,36.185158


In [ ]:
assembler= VectorAssembler(
    inputCols=['mach','aoa','CoordinateX', 'CoordinateY','CoordinateZ' ],outputCol='features')


output= assembler.transform(df)
output.show

<bound method DataFrame.show of DataFrame[mach: double, aoa: double, beta: double, CoordinateX: double, CoordinateY: double, CoordinateZ: double, X Component Wall Shear: double, Pressure: double, Pressure Coefficient: double, Y Component Wall Shear: double, Z Component Wall Shear: double, features: vector]>

In [ ]:
#final data consist of features and label which is medv.
final_data=output.select('features','Pressure')
#splitting data into train and test
train_data,test_data=final_data.randomSplit([0.8,0.2])
#creating an object of class LinearRegression
#object takes features and label as input arguments
#rf = GBTRegressor(featuresCol='features',labelCol='Pressure',maxiter=10)
rf = GBTRegressor(featuresCol='features',labelCol='Pressure',maxIter=10)

#lr=LinearRegression(featuresCol='features',labelCol='Pressure')

In [ ]:

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[ rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(train_data)

# Make predictions.
predictions = model.transform(test_data)

# Select example rows to display.
predictions.select("prediction", "Pressure", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="Pressure", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[0]
print(rfModel)  # summary only

+-------------------+-------------------+--------------------+
|         prediction|           Pressure|            features|
+-------------------+-------------------+--------------------+
|-386.41866497791693|-178.48419189453125|[0.20000000298023...|
|-222.79916078771436|-198.03683471679688|[0.20000000298023...|
|-222.79916078771436|-207.58035278320312|[0.20000000298023...|
|-222.79916078771436| -245.3154754638672|[0.20000000298023...|
|-222.79916078771436| -186.2625732421875|[0.20000000298023...|
+-------------------+-------------------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 4108.55
GBTRegressionModel: uid=GBTRegressor_84336e3b8fdc, numTrees=10, numFeatures=5


In [ ]:
#pass train_data to train model
lrModel=lr.fit(train_data)
# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
#print("numIterations: %d" % trainingSummary.totalIterations)
#print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
#trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

NameError: ignored